Importar librerias

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_log_error

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeRegressor


Cargar dataset original

In [2]:
PATH = "./Housing Dreams/"
DATASET = "house_train_raw.csv"

def load_dataset(path,file):
    csv_path = os.path.join(path,file)
    return pd.read_csv(csv_path)

df  = load_dataset(PATH,DATASET)

In [3]:
df.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Separar set de Train y Test

- Muestreo estratificado

In [4]:
split = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=0)

for train_index, test_index in split.split(df, df["OverallQual"]):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

In [5]:
X_train = train_set.drop('SalePrice',axis = 1)
y_train = train_set['SalePrice'].copy()

X_test = test_set.drop('SalePrice', axis=1)
y_test = test_set['SalePrice'].copy()

Data cleaning

- Definir funciones de preprocesamiento para incluirlas en el pipeline.
- Drop de columnas con muchos nulos o que no aporten.
- MSSubClass: convertir a `object` dado que es una variable categórica nominal (no numérica). Se hace con Transformer propio.
- Imputar valores faltantes (`SimpleImputer`).

In [6]:
class ChangeTypeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None, data_type='object'):
        self.columns = columns
        self.data_type = data_type
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        if self.columns:
            for col in self.columns:
                X[col] = X[col].astype(self.data_type)
        return X   

Identificación de tipos de columnas

In [7]:
cols_to_drop = ['Id','Alley','MiscFeature','PoolQC','MasVnrArea','GarageYrBlt','MoSold','Fence']

# Para cambiar de numerico a categorico.
cols_to_object = ['MSSubClass']

numerical_cols = [col for col in X_train.columns if ((X_train[col].dtype in ('int64','float64')) and (col not in (cols_to_drop + cols_to_object)))]

categorical_cols = [col for col in X_train.columns if col not in (numerical_cols + cols_to_drop)]

Pipelines

In [8]:
type_pipeline = Pipeline([
    ('change_type', ChangeTypeTransformer(columns = cols_to_object, data_type='object')),
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

full_pipeline = ColumnTransformer([
    ('change_data_type', type_pipeline, cols_to_object),
    ('cat', OneHotEncoder(handle_unknown = 'ignore'), categorical_cols),
    ('num', num_pipeline, numerical_cols)
])

Entrenar modelo

- Regresión lineal

In [9]:
X_train = train_set.drop('SalePrice',axis = 1).drop(columns = cols_to_drop)
y_train = train_set['SalePrice'].copy()

X_test = test_set.drop('SalePrice', axis=1).drop(columns = cols_to_drop)
y_test = test_set['SalePrice'].copy()

In [10]:
X = full_pipeline.fit_transform(X_train)

In [11]:
model_reg = DummyRegressor()
model_reg.fit(X, y_train)

DummyRegressor()

In [12]:
y_predictions = model_reg.predict(X)

In [13]:
model_rmsle = mean_squared_log_error(y_predictions,y_train, squared=False)

print("Error logaritmico: ", model_rmsle)

Error logaritmico:  0.4029154708369364


Predecir para el conjunto Test

In [14]:
X_test_prepared = full_pipeline.transform(X_test)

final_predictions = model_reg.predict(X_test_prepared)

In [15]:
model_rmsle_test = mean_squared_log_error(final_predictions,y_test, squared=False)

print("Error logaritmico (test): ", model_rmsle_test)

Error logaritmico (test):  0.42394065828945154


Aplicar modelo a `houses_test_raw.csv`

In [16]:
DATASET_TEST = 'houses_test_raw.csv' 
df_test  = load_dataset(PATH,DATASET_TEST)

In [18]:
X_test_final = df_test
X_test_final_prepared = full_pipeline.transform(X_test_final)

y_final = model_reg.predict(X_test_final_prepared)

In [ ]:
y_final

array([115732.69887083, 229289.18341797, 184458.95311593, ...,
       176252.49636359, 117928.97544166, 228568.27192047])

In [ ]:
predictions = pd.DataFrame(y_final, columns=['pred'])

In [ ]:
predictions.head()

,pred
0,115732.698871
1,229289.183418
2,184458.953116
3,201254.144121
4,200810.874157


In [ ]:
#predictions.to_csv('pred_test.csv', index=False)